```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    "" 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

### 도서 정보 추출

In [ ]:
# import tools.ProcessFiles as pf
# excel = pf.get_excel_data(3) # i차 분석
# excel

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [1]:
import pandas as pd
import os

base_path = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 1

excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3)[['관리번호_old', '관리번호', 'ISBN', '도서명', '분류']]

In [7]:
# Fixed version - using Python file operations instead of sed
import shutil
from tqdm import tqdm

for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
   fn = str(fn)
   if '/' in lv: 
      lv = lv.replace("/", '_')
   
   if 'Lv' not in lv:
      continue

   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
      continue
   else:
      os.system(f"cp /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영/{fn}.zip /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/{lv}/{tn}.zip && cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

      file_path = f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/{lv}/{tn}/{tn}.json"
      try:
         with open(file_path, 'r', encoding='utf-8') as f:
               content = f.read()
         
         # Replace the old filename with new filename
         updated_content = content.replace(str(fn), str(tn))
         
         with open(file_path, 'w', encoding='utf-8') as f:
               f.write(updated_content)
               
         print(f"Updated {fn} to {tn} in {file_path}")
      except Exception as e:
         print(f"Error processing {fn} to {tn}: {e}")

         os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

0it [00:00, ?it/s]mkdir: SS0319: File exists
1it [00:30, 30.57s/it]

Updated 225933255 to SS0319 in /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0319/SS0319.json


mkdir: SS0320: File exists
2it [00:49, 23.80s/it]

Updated 239373741 to SS0320 in /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0320/SS0320.json


mkdir: SS0321: File exists
3it [02:06, 48.00s/it]

...

Error processing 362874088 to SS0422: [Errno 2] No such file or directory: '/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0422/SS0422.json'
Archive:  SS0422.zip


cp: /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0423.zip: fcopyfile failed: Operation timed out
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of SS0423.zip or
        SS0423.zip.zip, and cannot find SS0423.zip.ZIP, period.
105it [2:50:08, 55.59s/it]

Error processing 364690338 to SS0423: [Errno 2] No such file or directory: '/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0423/SS0423.json'
Archive:  SS0423.zip


cp: /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0424.zip: fcopyfile failed: Operation timed out
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of SS0424.zip or
        SS0424.zip.zip, and cannot find SS0424.zip.ZIP, period.
106it [2:50:20, 96.42s/it]

Error processing 346764368 to SS0424: [Errno 2] No such file or directory: '/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0424/SS0424.json'
Archive:  SS0424.zip


In [ ]:
# len(excel_analy[excel_analy['분류'] == 'Lv3/4'])

56

In [ ]:
import os

# 업로드
for ss in file:
    print(ss)
    os.system(f"scp -r /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/3C/Lv3_4/{ss} yjmoon@oci004:/home/yjmoon/FINAL/{ss}")

In [ ]:
for fn, tn, lv in zip(excel_analy['관리번호'], excel_analy['관리번호_new'], excel_analy['분류']):
    fn = str(fn)
    if '/' in lv: 
       lv = lv.replace("/", '_')
#     print(fn, tn, lv)
    
    dir = os.path.join(pf.FINAL_DATA_PATH, pf.CYCLE_PATH[1], lv)
    os.system(f'cd {dir} && mv {fn}_low {tn}_low && cd {tn} &&'+f' for file in *{fn}*; do mv "$file" "$'+'{file//'+fn+'/'+tn+'}"; done')
    print(os.path.join(dir, fn))
    # break

In [ ]:
import tools.ProcessFiles as pf

pf.move_jsons(3, pf.ORIGINAL_DATA_PATH)

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

## 전체 돌리기

In [ ]:
import os, json
from typing import List, Dict, Any
import tools.ProcessFiles as pf
import tools.ProcessLv2 as pl2

json_files = pf.get_filelist(1)
json_files

In [ ]:
# import os, json, shutil
# import tools.ProcessFiles as pf

# i = 3
# # 폴더 만들어 옮기기
# excel = pf.get_excel_data(i)
# FINAL_DATA_PATH = os.path.join("/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter", '3C')
# ORIGINAL_DATA_PATH = os.path.join("/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/", '3C')

# json_files = pf.get_filelist(i)

# for j in json_files:
#     new_path = j.replace(ORIGINAL_DATA_PATH, FINAL_DATA_PATH)
#     try:
#         shutil.copy(j, new_path)
#     except FileNotFoundError:
#         os.makedirs(os.path.dirname(new_path), exist_ok=True)
#         shutil.copy(j, new_path)

In [ ]:
import json
import tools.ProcessLv2 as pl2

# for j in json_files:
if True:
    j = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C/Lv2/SS0017_low/SS0017.json'
    INPUT_PATH = j
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    with open(BACKUP_PATH, "w", encoding="utf-8") as f:
        json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # break
    # id = os.path.splitext(os.path.basename(j))[0]
    # try:
    #     new = {
    #         'file_id': id,
    #         'ISBN': str(excel.loc[id, 'ISBN']),
    #         'title': excel.loc[id, '도서명'],
    #         'cat1_domain': excel.loc[id, '코퍼스 1분류'],
    #         'cat2_sub': excel.loc[id, '코퍼스 2분류'],
    #         'cat3_specific': excel.loc[id, '비고'],
    #         'pub_date': str(excel.loc[id, '출판일'])[:10],
    #         # 'contents': [],
    #         'contents': origin['contents']
    #     }
    # except:
    #     new = origin
    
    new = pl2.fill_chapter(origin)
    # new = pl2.merge_paragraphs(origin)
    # new = pl2.erase_page(origin, 1)
    # new = pl2.format_change(3,origin)
    # new = origin

    # for i in range(len(new['contents'])):
        # contents = new['contents'][i]
        # # c = extract_qna(contents)
        # page_contents = pl2.remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # # c = contents

    with open(INPUT_PATH.replace(".json", ".json"), "w", encoding="utf-8") as f:
    # with open(INPUT_PATH.replace(".json", "_new.json"), "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)